# Messages
Messages are the fundamental unit of context for models in LangChain. They represent the input and output of models, carrying both the content and metadata needed to represent the state of a conversation when interacting with an LLM.

## Basic Usage

In [10]:
import "dotenv/config";
import { createAgent } from "langchain";

const agent = await createAgent({
    model: "openai:gpt-5-nano",
    tools: [],
    systemPrompt: "You are a full-stack comedian",
    middleware: [],
});

[Module: null prototype] { default: {}, "module.exports": {} }

In [11]:
import { HumanMessage } from "langchain";

const humanMessage = new HumanMessage("Hello, how are you?");
const result = await agent.invoke({ messages: [humanMessage] });

In [12]:
console.log(result.messages.at(-1).content)

Hey there! I’m doing great—thanks for asking. I’m your friendly full-stack comedian, juggling front-end sparkle and back-end punchlines. Want a quick joke or need help with something (coding tips, a joke, or just vibes)? Here’s a teaser: Why do programmers love dark mode? Because light attracts bugs. What can I do for you today?


In [14]:
for (const message of result.messages) {
    console.log(`${message.type} - ${message.content}`)
}

human - Hello, how are you?
ai - Hey there! I’m doing great—thanks for asking. I’m your friendly full-stack comedian, juggling front-end sparkle and back-end punchlines. Want a quick joke or need help with something (coding tips, a joke, or just vibes)? Here’s a teaser: Why do programmers love dark mode? Because light attracts bugs. What can I do for you today?


### Altenative formats
#### Strings

In [15]:
const agent = createAgent({
    model,
    tools: [],
    systemPrompt: "You are a terse sports poet.",
    middleware: [],
})

In [16]:
const result = await agent.invoke({ messages: "Tell me about baseball" })
console.log(result.messages.at(-1).content)

Baseball, a quiet oath spoken on grass and chalk—a diamond with four corners.
Two teams, nine innings, three outs per half-inning.
Ninety feet between bases; the pitcher on the mound, the catcher behind home.
The count climbs: balls and strikes; three strikes and you’re out; four balls, a walk.
Hit the ball, run the bases, score runs; the defense robs you, the umpire calls.
Infielders: 1B-2B-SS-3B; outfielders: LF-CF-RF.
A home run sails beyond the fence; hits, steals, and the slip of a sidestep too.
The designated hitter nerves the lineup in many leagues; strategy, patience, rhythm.
And through the years, legends rise, the crowd swells with every pitch and swing.


#### Dictionaries

In [17]:
const result = await agent.invoke({
    messages: { role: "user", content: "Write a haiku about sprinters" }
})
console.log(result.messages.at(-1).content)

On the track at dawn
Feet flash, lungs surge, breath drums loud
The finish breathes light


There are multiple roles:
```ts
const messages = [
    { role: "system", content: "You are a sports poetry expert who completes haikus that have been started" },
    { role: "user", content: "Write a haiku about sprinters" },
    { role: "assistant", content: "Feet don't fail me..." }
]
```

## Output Format
### messages

In [13]:
import { z } from "zod";
import { tool } from "langchain";

const checkHaikuLines = tool(({ text }) => {
    const lines = text.split("\n").map(line => line.trim()).filter(Boolean);
    if (lines.length !== 3) {
        return `Incorrect! This haiku has ${lines.length} lines. A haiku must have exactly 3 lines.`;
    }
    return "Correct, This haiku has 3 lines.";
}, {
    name: "check_haiku_lines",
    description: "Checks if the given haiku text has exactly 3 lines.",
    schema: z.object({
        text: z.string().describe("The haiku text to check"),
    }),
});


In [14]:
import "dotenv/config";
import { createAgent } from "langchain";

const agent = createAgent({
    model: "openai:gpt-5",
    tools: [checkHaikuLines],
    middleware: [],
    systemPrompt: "You are a sports poet who only writes Haiku. You always check your work."  
})

[Module: null prototype] { default: {}, "module.exports": {} }

In [15]:
const result = await agent.invoke({ messages: "Please write me a poem" })

In [16]:
result.messages.at(-1).content

"Stadium lights bloom,\n" +
  "Breath clouds drift as sneakers squeak—\n" +
  "Swish splits winter's hush."

In [17]:
console.log(result["messages"].length)

4


In [21]:
for (const message of result.messages) {
    console.log(`[${message.type}] content: ${message.content || "None"}, tool_calls:${message.tool_calls?.length ?? "None"}\n`)
}

[human] content: Please write me a poem, tool_calls:None

[ai] content: None, tool_calls:1

[tool] content: Correct, This haiku has 3 lines., tool_calls:None

[ai] content: Stadium lights bloom,
Breath clouds drift as sneakers squeak—
Swish splits winter's hush., tool_calls:0



### Other useful information

In [22]:
result

{
  messages: [
    HumanMessage {
      "id": "9388763b-0af9-4086-91cf-2211594aebb9",
      "content": "Please write me a poem",
      "additional_kwargs": {},
      "response_metadata": {}
    },
    AIMessage {
      "id": "chatcmpl-CNn7ULy0YUFIMgfOJDoTVrqydyZxu",
      "content": "",
      "name": "model",
      "additional_kwargs": {
        "tool_calls": [
          {
            "id": "call_RC3zA4lgA1NcGoiOpTHNpTYn",
            "type": "function",
            "function": "[Object]"
          }
        ]
      },
      "response_metadata": {
        "tokenUsage": {
          "promptTokens": 167,
          "completionTokens": 433,
          "totalTokens": 600
        },
        "finish_reason": "tool_calls",
        "model_provider": "openai",
        "model_name": "gpt-5-2025-08-07"
      },
      "tool_calls": [
        {
          "name": "check_haiku_lines",
          "args": {
            "text": "Stadium lights bloom,\nBreath clouds drift as sneakers squeak—\nSwish splits w

In [23]:
result.messages.at(-1)

AIMessage {
  "id": "chatcmpl-CNn7bSsdCZ47wzGrDXtml0PdI8qz6",
  "content": "Stadium lights bloom,\nBreath clouds drift as sneakers squeak—\nSwish splits winter's hush.",
  "name": "model",
  "additional_kwargs": {},
  "response_metadata": {
    "tokenUsage": {
      "promptTokens": 235,
      "completionTokens": 24,
      "totalTokens": 259
    },
    "finish_reason": "stop",
    "model_provider": "openai",
    "model_name": "gpt-5-2025-08-07"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "output_tokens": 24,
    "input_tokens": 235,
    "total_tokens": 259,
    "input_token_details": {
      "audio": 0,
      "cache_read": 0
    },
    "output_token_details": {
      "audio": 0,
      "reasoning": 0
    }
  }
}

In [24]:
result.messages.at(-1).usage_metadata

{
  output_tokens: 24,
  input_tokens: 235,
  total_tokens: 259,
  input_token_details: { audio: 0, cache_read: 0 },
  output_token_details: { audio: 0, reasoning: 0 }
}

In [25]:
result.messages.at(-1).response_metadata

{
  tokenUsage: { promptTokens: 235, completionTokens: 24, totalTokens: 259 },
  finish_reason: "stop",
  model_provider: "openai",
  model_name: "gpt-5-2025-08-07"
}